# SparkSQL and DataFrames 

<a href = "http://yogen.io"><img src="http://yogen.io/assets/logo.svg" alt="yogen" style="width: 200px; float: right;"/></a>

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://apache.uvigo.es/spark/spark-2.4.6/spark-2.4.6-bin-hadoop2.7.tgz
!tar -xf spark-2.4.6-bin-hadoop2.7.tgz
!pip install -q findspark pyspark==2.4.6
import os
import findspark
from pyspark.sql import SparkSession
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.6-bin-hadoop2.7"
findspark.init()
spark = SparkSession.builder.master("local[*]").getOrCreate()

     |████████████████████████████████| 218.4MB 61kB/s 
     |████████████████████████████████| 204kB 44.9MB/s 


## RDDs, DataSets, and DataFrames

RDDs are the original interface for Spark programming.

DataFrames were introduced in 1.3

Datasets were introduced in 1.6, and unified with DataFrames in 2.0

### Advantages of DataFrames:

from https://www.datacamp.com/community/tutorials/apache-spark-python:

> More specifically, the performance improvements are due to two things, which you’ll often come across when you’re reading up DataFrames: custom memory management (project Tungsten), which will make sure that your Spark jobs much faster given CPU constraints, and optimized execution plans (Catalyst optimizer), of which the logical plan of the DataFrame is a part.

## SparkSQL and DataFrames 


pyspark does not have the Dataset API, which is available only if you use Spark from a statically typed language: Scala or Java.

From https://spark.apache.org/docs/2.4.4/sql-programming-guide.html

> A DataFrame is a Dataset organized into named columns. It is conceptually equivalent to a table in a relational database or a data frame in R/Python, but with richer optimizations under the hood. DataFrames can be constructed from a wide array of sources such as: structured data files, tables in Hive, external databases, or existing RDDs. The DataFrame API is available in Scala, Java, Python, and R. In Scala and Java, a DataFrame is represented by a Dataset of Rows. In the Scala API, DataFrame is simply a type alias of Dataset[Row]. While, in Java API, users need to use Dataset&lt;Row> to represent a DataFrame.


### The pyspark.sql module

Important classes of Spark SQL and DataFrames:

* `pyspark.sql.SparkSession` Main entry point for DataFrame and SQL functionality.

* `pyspark.sql.DataFrame` A distributed collection of data grouped into named columns.

* `pyspark.sql.Column` A column expression in a DataFrame.

* `pyspark.sql.Row` A row of data in a DataFrame.

* `pyspark.sql.GroupedData` Aggregation methods, returned by DataFrame.groupBy().

* `pyspark.sql.DataFrameNaFunctions` Methods for handling missing data (null values).

* `pyspark.sql.DataFrameStatFunctions` Methods for statistics functionality.

* `pyspark.sql.functions` List of built-in functions available for DataFrame.

* `pyspark.sql.types` List of data types available.

* `pyspark.sql.Window` For working with window functions.

http://spark.apache.org/docs/2.4.4/api/python/pyspark.sql.html

https://spark.apache.org/docs/2.4.4/sql-programming-guide.html

## SparkSession

The traditional way to interact with Spark is the SparkContext. In the notebooks we get that from the pyspark driver.

From 2.0 we can use SparkSession to replace SparkConf, SparkContext and SQLContext

In [2]:
spark = SparkSession.builder.master("local[*]").getOrCreate()

#### Passing other options to spark session:
    
    

In [3]:
spark = SparkSession.builder.config('thiscanbeanykey', 'thiscanbeanyvalue').master("local[*]").getOrCreate()

We can check option values in the resulting session like this:

In [5]:
spark.sparkContext.getConf().getAll()

[('spark.driver.host', '69700d245be9'),
 ('spark.rdd.compress', 'True'),
 ('thiscanbeanykey', 'thiscanbeanyvalue'),
 ('spark.driver.port', '40691'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.app.id', 'local-1592587182145'),
 ('spark.master', 'local[*]'),
 ('spark.executor.id', 'driver'),
 ('spark.submit.deployMode', 'client'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.app.name', 'pyspark-shell')]

### Creating DataFrames

SparkSession.createDataFrame: from an RDD, a list or a pandas.DataFrame.

In [6]:
help(spark.createDataFrame)

Help on method createDataFrame in module pyspark.sql.session:

createDataFrame(data, schema=None, samplingRatio=None, verifySchema=True) method of pyspark.sql.session.SparkSession instance
    Creates a :class:`DataFrame` from an :class:`RDD`, a list or a :class:`pandas.DataFrame`.
    
    When ``schema`` is a list of column names, the type of each column
    will be inferred from ``data``.
    
    When ``schema`` is ``None``, it will try to infer the schema (column names and types)
    from ``data``, which should be an RDD of either :class:`Row`,
    :class:`namedtuple`, or :class:`dict`.
    
    When ``schema`` is :class:`pyspark.sql.types.DataType` or a datatype string, it must match
    the real data, or an exception will be thrown at runtime. If the given schema is not
    :class:`pyspark.sql.types.StructType`, it will be wrapped into a
    :class:`pyspark.sql.types.StructType` as its only field, and the field name will be "value".
    Each record will also be wrapped into a tu

In [12]:
import random

random.seed(42)
races = random.choices(['elf', 'hobbit', 'troll'], k=15)
ids = range(15)
races

['hobbit',
 'elf',
 'elf',
 'elf',
 'troll',
 'troll',
 'troll',
 'elf',
 'hobbit',
 'elf',
 'elf',
 'hobbit',
 'elf',
 'elf',
 'hobbit']

In [16]:
data = list(zip(ids, races))
data

[(0, 'hobbit'),
 (1, 'elf'),
 (2, 'elf'),
 (3, 'elf'),
 (4, 'troll'),
 (5, 'troll'),
 (6, 'troll'),
 (7, 'elf'),
 (8, 'hobbit'),
 (9, 'elf'),
 (10, 'elf'),
 (11, 'hobbit'),
 (12, 'elf'),
 (13, 'elf'),
 (14, 'hobbit')]

In [17]:
df = spark.createDataFrame(data)
df

DataFrame[_1: bigint, _2: string]

In [18]:
df.take(5)

[Row(_1=0, _2='hobbit'),
 Row(_1=1, _2='elf'),
 Row(_1=2, _2='elf'),
 Row(_1=3, _2='elf'),
 Row(_1=4, _2='troll')]

In [19]:
df.rdd

MapPartitionsRDD[11] at javaToPython at NativeMethodAccessorImpl.java:0

In [20]:
df.first()

Row(_1=0, _2='hobbit')

In [22]:
from pyspark.sql import Row

Row(hairstyle='bald', height=1.8, name='Bruce Willis')

Row(hairstyle='bald', height=1.8, name='Bruce Willis')

In [25]:
data = [ Row(id_=id_, race=race) for id_, race in zip(ids, races) ]
data

[Row(id_=0, race='hobbit'),
 Row(id_=1, race='elf'),
 Row(id_=2, race='elf'),
 Row(id_=3, race='elf'),
 Row(id_=4, race='troll'),
 Row(id_=5, race='troll'),
 Row(id_=6, race='troll'),
 Row(id_=7, race='elf'),
 Row(id_=8, race='hobbit'),
 Row(id_=9, race='elf'),
 Row(id_=10, race='elf'),
 Row(id_=11, race='hobbit'),
 Row(id_=12, race='elf'),
 Row(id_=13, race='elf'),
 Row(id_=14, race='hobbit')]

In [27]:
df = spark.createDataFrame(data)
df

DataFrame[id_: bigint, race: string]

### Creating DataFrames

* From RDDs
* from Hive tables
* From Spark sources: parquet (default), json, jdbc, orc, libsvm, csv, text


#### From RDDs

In [38]:
coupon_rdd = spark.sparkContext.textFile('coupon150720.csv').map(lambda line: line.split(','))
coupon_rdd.take(2)

[['79062005698500',
  '1',
  'MAA',
  'AUH',
  '9W',
  '9W',
  '56.79',
  'USD',
  '1',
  'H',
  'H',
  '0526',
  '150904',
  'OK',
  'IAF0'],
 ['79062005698500',
  '2',
  'AUH',
  'CDG',
  '9W',
  '9W',
  '84.34',
  'USD',
  '1',
  'H',
  'H',
  '6120',
  '150905',
  'OK',
  'IAF0']]

In [39]:
spark.createDataFrame(coupon_rdd)

DataFrame[_1: string, _2: string, _3: string, _4: string, _5: string, _6: string, _7: string, _8: string, _9: string, _10: string, _11: string, _12: string, _13: string, _14: string, _15: string]

### Inferring and specifying schemas

#### Fully specifying a schema

We need to create a `StructType` composed of `StructField`s. each of those specifies afiled with name, type and `nullable` properties. 

#### From csv files

We can either read them directly into dataframes or read them as RDDs and transform that into a DataFrame. This second way will be very useful if we have unstructured data like web server logs.

#### From other types of data

Apache Parquet is a free and open-source column-oriented data store of the Apache Hadoop ecosystem. It is similar to the other columnar storage file formats available in Hadoop namely RCFile and Optimized RCFile. It is compatible with most of the data processing frameworks in the Hadoop environment.

### Basic operations with DataFrames

In [42]:
x = df.show(5)

+---+------+
|id_|  race|
+---+------+
|  0|hobbit|
|  1|   elf|
|  2|   elf|
|  3|   elf|
|  4| troll|
+---+------+
only showing top 5 rows



In [43]:
print(x)

None


### Filtering and selecting

Syntax inspired in SQL.

In [45]:
df.select('race').show()

+------+
|  race|
+------+
|hobbit|
|   elf|
|   elf|
|   elf|
| troll|
| troll|
| troll|
|   elf|
|hobbit|
|   elf|
|   elf|
|hobbit|
|   elf|
|   elf|
|hobbit|
+------+



In [46]:
df.select('race')

DataFrame[race: string]

If we want to filter, we will need to build an instance of `Column`, using square bracket notation.

In [49]:
df['race'].take()

TypeError: ignored

In [51]:
df.filter(df['race']=='elf').show()

+---+----+
|id_|race|
+---+----+
|  1| elf|
|  2| elf|
|  3| elf|
|  7| elf|
|  9| elf|
| 10| elf|
| 12| elf|
| 13| elf|
+---+----+



In [55]:
df.filter(df['id_'] < 5).show()

+---+------+
|id_|  race|
+---+------+
|  0|hobbit|
|  1|   elf|
|  2|   elf|
|  3|   elf|
|  4| troll|
+---+------+



In [56]:
df.filter('id_' < 5).show()

TypeError: ignored

That's because a comparison between str and int will error out, so spark will not even get the chance to infer to which column we are referring.

In [57]:
'id_' < 5

TypeError: ignored

`where` is exactly synonimous with `filter`

In [58]:
df.where(df['id_'] < 5).show()

+---+------+
|id_|  race|
+---+------+
|  0|hobbit|
|  1|   elf|
|  2|   elf|
|  3|   elf|
|  4| troll|
+---+------+



A column is quite different to a Pandas Series. It is just a reference to a column, and can only be used to construct sparkSQL expressions (select, where...). It can't be collected or taken as a one-dimensional sequence:

#### Exercise

Extract all mythical being ids which correspond to hobbits

In [62]:
df.filter(df['race'] == 'hobbit').select('id_').show()

+---+
|id_|
+---+
|  0|
|  8|
| 11|
| 14|
+---+



### Adding columns

Dataframes are immutable, since they are built on top of RDDs, so we can not assign to them. We need to create new DataFrames with the appropriate columns.

In [64]:
df[0] = 'elf'

TypeError: ignored

In [66]:
df['id_'] ** 2

Column<b'POWER(id_, 2)'>

In [68]:
df2 = df.withColumn('newcol', df['id_'] ** 2)
df2.show()

+---+------+------+
|id_|  race|newcol|
+---+------+------+
|  0|hobbit|   0.0|
|  1|   elf|   1.0|
|  2|   elf|   4.0|
|  3|   elf|   9.0|
|  4| troll|  16.0|
|  5| troll|  25.0|
|  6| troll|  36.0|
|  7|   elf|  49.0|
|  8|hobbit|  64.0|
|  9|   elf|  81.0|
| 10|   elf| 100.0|
| 11|hobbit| 121.0|
| 12|   elf| 144.0|
| 13|   elf| 169.0|
| 14|hobbit| 196.0|
+---+------+------+



In [69]:
df.select('id_', df['id_'] * 100)

DataFrame[id_: bigint, (id_ * 100): bigint]

### User defined functions

There are many useful functions in pyspark.sql.functions. These work on columns, that is, they are vectorial.

We can write User Defined Functions (`udf`s), which allow us to "vectorize" operations: write a standard function to process single elements, then build a udf with that that works on columns in a DataFrame, like a SQL function.

In [74]:
from pyspark.sql import functions as f

df.select('id_',
          'race',
          f.tanh('id_'),
          f.log1p('id_'))


DataFrame[id_: bigint, race: string, TANH(id_): double, LOG1P(id_): double]

In [78]:
import math

math.factorial(5)

120

In [80]:
math.factorial('id_')

TypeError: ignored

In [81]:
math.factorial(df['id_'])

TypeError: ignored

This errors out because 

```python
math.factorial
```

is not a udf: it doesn't know how to work with strings or Column objects:

But we can transform it into a udf:

In [84]:
factorial_udf = f.udf(math.factorial) 
factorial_udf

<function math.factorial>

In [85]:
factorial_udf('id_')

Column<b'factorial(id_)'>

We can do the same with any function we dream up:

In [91]:
panoli_malote = f.udf(lambda: random.choice(['panoli', 'malote']))
dice_12 = f.udf(lambda: random.randrange(1,13))
baddies = df.select('*',
                    panoli_malote(),
                    dice_12())
baddies.show()

+---+------+----------+----------+
|id_|  race|<lambda>()|<lambda>()|
+---+------+----------+----------+
|  0|hobbit|    malote|         6|
|  1|   elf|    panoli|         5|
|  2|   elf|    panoli|         9|
|  3|   elf|    panoli|         7|
|  4| troll|    panoli|         4|
|  5| troll|    panoli|         8|
|  6| troll|    panoli|         6|
|  7|   elf|    panoli|         3|
|  8|hobbit|    malote|         4|
|  9|   elf|    malote|         9|
| 10|   elf|    panoli|        10|
| 11|hobbit|    malote|         7|
| 12|   elf|    panoli|         5|
| 13|   elf|    panoli|         8|
| 14|hobbit|    panoli|         3|
+---+------+----------+----------+



If we want the resulting columns to be of a particular type, we need to specify the return type. This is because in Python return types can not be inferred.

In [94]:
from pyspark.sql import types

panoli_malote = f.udf(lambda: random.choice(['panoli', 'malote']))
dice_12 = f.udf(lambda: random.randrange(1,13), returnType=types.IntegerType())
baddies = df.select('*',
                    panoli_malote(),
                    dice_12())
baddies

DataFrame[id_: bigint, race: string, <lambda>(): string, <lambda>(): int]

Think about this function: what is its return type?

In [ ]:
def something(a, b):
  return a + b

In [99]:
from pyspark.sql import types

panoli_malote = f.udf(lambda: random.choice(['panoli', 'malote']))
dice_12 = f.udf(lambda: random.randrange(1,13))
baddies = df.select('*',
                    panoli_malote().alias('alignment'),
                    dice_12().cast(types.IntegerType()).alias('dexterity'),
                    dice_12().cast(types.IntegerType()).alias('strength'))
baddies.show()

+---+------+---------+---------+--------+
|id_|  race|alignment|dexterity|strength|
+---+------+---------+---------+--------+
|  0|hobbit|   malote|       11|       9|
|  1|   elf|   panoli|        1|       8|
|  2|   elf|   panoli|        7|      10|
|  3|   elf|   malote|       12|      12|
|  4| troll|   panoli|        3|       9|
|  5| troll|   malote|        4|       9|
|  6| troll|   malote|        4|       4|
|  7|   elf|   panoli|       12|       7|
|  8|hobbit|   panoli|        7|       7|
|  9|   elf|   panoli|        4|      11|
| 10|   elf|   panoli|       12|       7|
| 11|hobbit|   panoli|       11|       5|
| 12|   elf|   malote|        1|       9|
| 13|   elf|   panoli|        2|      12|
| 14|hobbit|   panoli|        8|       1|
+---+------+---------+---------+--------+



#### Exercise: 

Create an 'hp' field in our df. make it 30000 for hobbits, 40000 for elves and 70000 for trolls.





In [107]:
race = 'hobbit'

def hp_from_race(race):

  if race == 'hobbit': 
    return 30000
  elif race == 'elf':
    return 40000
  elif race == 'troll':
    return 70000

hp_from_race(race)

hp_udf = f.udf(hp_from_race, returnType=types.IntegerType())

with_hp = baddies.withColumn('hp', hp_udf('race'))
with_hp

DataFrame[id_: bigint, race: string, alignment: string, dexterity: int, strength: int, hp: int]

If we have a column that is not the desired type, we can convert it with `cast`.

### Summary statistics

https://databricks.com/blog/2015/06/02/statistical-and-mathematical-functions-with-dataframes-in-spark.html

In [108]:
with_hp.corr('hp', 'dexterity')

-0.16485986761022522

In [110]:
with_hp.stat.cov('dexterity', 'strength')

-0.5714285714285713

### .crosstab()

Crosstab returns the contingency table for two columns, as a DataFrame.

In [128]:
with_hp.cache().crosstab('alignment', 'race').show()

+--------------+---+------+-----+
|alignment_race|elf|hobbit|troll|
+--------------+---+------+-----+
|        malote|  5|     1|    2|
|        panoli|  3|     3|    1|
+--------------+---+------+-----+



### Grouping

Grouping works very similarly to Pandas: executing groupby (or groupBy) on a DataFrame will return an object (a GroupedData) that can then be aggregated to obtain the results.

In [130]:
gd = with_hp.groupby('race')
gd

GroupedData has several aggregation functions defined:

In [133]:
gd.mean('strength').show()

+------+-----------------+
|  race|    avg(strength)|
+------+-----------------+
| troll|6.666666666666667|
|hobbit|             7.25|
|   elf|             8.75|
+------+-----------------+



We can do several aggregations in a single step, with a number of different syntaxes:

In [142]:
gd.agg({'strength' : 'avg', 'dexterity': 'stddev'}).show()

+------+------------------+-----------------+
|  race| stddev(dexterity)|    avg(strength)|
+------+------------------+-----------------+
| troll| 4.618802153517007|6.666666666666667|
|hobbit|1.2909944487358056|             7.25|
|   elf| 3.845219666769935|             8.75|
+------+------------------+-----------------+



In [145]:
gd.agg(f.mean('dexterity'), 
       f.stddev('dexterity'),
       f.max('dexterity'),
       f.min('dexterity')).show()

+------+-----------------+----------------------+--------------+--------------+
|  race|   avg(dexterity)|stddev_samp(dexterity)|max(dexterity)|min(dexterity)|
+------+-----------------+----------------------+--------------+--------------+
| troll|6.666666666666667|     4.618802153517007|            12|             4|
|hobbit|              8.5|    1.2909944487358056|            10|             7|
|   elf|             6.75|     3.845219666769935|            12|             1|
+------+-----------------+----------------------+--------------+--------------+



### Intersections

Ver much like SQL joins. We can specify the columns and the join method (left, right, inner, outer) or we can let Spark infer them.

In [146]:
stats = gd.agg(f.mean('dexterity'), 
               f.stddev('dexterity'),
               f.max('dexterity'),
               f.min('dexterity'))


In [148]:
with_hp.join(stats).show()

AnalysisException: ignored

Spark refuses to do cross joins by default. To perform them, we can 

a) Allow then explicitly:

```python
session.conf.set("spark.sql.crossJoin.enabled", "true")
```

b) Specify the join criterion

```python
df4.join(new_df, on='id').show()
```

In [149]:
with_hp.join(stats, on='race').show()

+------+---+---------+---------+--------+-----+-----------------+----------------------+--------------+--------------+
|  race|id_|alignment|dexterity|strength|   hp|   avg(dexterity)|stddev_samp(dexterity)|max(dexterity)|min(dexterity)|
+------+---+---------+---------+--------+-----+-----------------+----------------------+--------------+--------------+
|hobbit|  0|   malote|        9|       4|30000|              8.5|    1.2909944487358056|            10|             7|
|   elf|  1|   malote|       10|      11|40000|             6.75|     3.845219666769935|            12|             1|
|   elf|  2|   malote|       12|      12|40000|             6.75|     3.845219666769935|            12|             1|
|   elf|  3|   panoli|        3|       9|40000|             6.75|     3.845219666769935|            12|             1|
| troll|  4|   malote|        4|       9|70000|6.666666666666667|     4.618802153517007|            12|             4|
| troll|  5|   malote|        4|       4|70000|6

#### Digression

We can monitor our running jobs and storage used at the Spark Web UI. We can get its url with sc.uiWebUrl.

StorageLevels represent how our DataFrame is cached: we can save the results of the computation up to that point, so that if we process several times the same data only the subsequent steps will be recomputed.

We can erase it with `unpersist`

#### Exercise

Calculate the [z-score](http://www.statisticshowto.com/probability-and-statistics/z-score/) of each creature's hp for their alignment


1) Calculate the mean and std of salary for each location

2) Annotate each employee with the stats corresponding to their location

3) Calculate the z-score

Note that we can build more complex boolean conditions for joining, as well as joining on columns that do not have the same name:

### Handling null values

## SQL querying

We need to register our DataFrame as a table in the SQL context in order to be able to query against it.

Once registered, we can perform queries as complex as we want.

## Interoperation with Pandas

Easy peasy. We can convert a spark DataFrame into a Pandas one, which will `collect` it, and viceversa, which will distribute it.

## Writing out


#### Exercise

Repeat the exercise from the previous notebook, but this time with DataFrames.

Get stats for all tickets with destination MAD from `coupons150720.csv`.

You will need to extract ticket amounts with destination MAD, and then calculate:

1. Total ticket amounts per origin
2. Top 10 airlines by average amount

1) Extract the fields you need (c0,c1,c2,c3,c4 and c6) into a dataframe with proper names and types

Remember, you want to calculate:

Total ticket amounts per origin

Top 10 airlines by average amount

2) Total ticket amounts per origin

3) Top 10 Airlines by average amount



## Further Reading

https://databricks.com/blog/2016/07/14/a-tale-of-three-apache-spark-apis-rdds-dataframes-and-datasets.html

https://www.datacamp.com/community/tutorials/apache-spark-python

https://spark.apache.org/docs/2.2.0/sql-programming-guide.html

https://ogirardot.wordpress.com/2015/05/29/rdds-are-the-new-bytecode-of-apache-spark/

https://stackoverflow.com/questions/36822224/what-are-the-pros-and-cons-of-parquet-format-compared-to-other-formats

https://s3.amazonaws.com/assets.datacamp.com/blog_assets/PySpark_SQL_Cheat_Sheet_Python.pdf